In [ ]:
#import des modules nécessaires au fonctionnement du programme 


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



import matplotlib.pyplot as plt
%matplotlib inline


from os import listdir
print(listdir("../input"))

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
# Any results you write to the current directory are saved as output.

In [ ]:
#ouverture des données 
train = pd.read_csv("../input/LANL-Earthquake-Prediction/train.csv", nrows=20000000,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})#lecture des 20M premiers indices pour les observations afin que ce soit plus facile à ouvrir
train.head(5)

train.rename({"acoustic_data": "signal", "time_to_failure": "quaketime"}, axis="columns", inplace=True)
train.head(5)
#Les valeurs sont placées dans un tableau où sont indexés le signal et le quaketime, à savoir le temps avant le prochain tremblement de terre, pour chacun des indices d'acquisition

# Introduction

L’objectif de ce projet est de prédire le temps restant jusqu’à ce que le prochain séisme ait lieu à un endroit donné en fonction d’un segment d’enregistrement sismique.

Pour ce faire, nous disposons d'un signal sismique artificiel enregistrés dans le temps et à une position fixe : ils constituent notre jeu de données, ce signal couvre plusieurs séismes. À noter que les temps d’enregistrement sont des échantillons discrets du temps. Par la suite, les temps d’enregistrement échantillonnés seront désignés comme les indices du signal.

## Visualisation du jeu de données

On visualise les données à partir desquelles nous allons réaliser l'apprentissage.

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(20,12))
ax[0].plot(train.index.values, train.quaketime.values, c="darkred")
ax[0].set_title("Temps avant le prochain séisme sur 20M d'indices")
ax[0].set_xlabel("Indice")
ax[0].set_ylabel("Temps avant le séisme en ms");
ax[1].plot(train.index.values, train.signal.values, c="mediumseagreen")
ax[1].set_title("Signal acoustique sur 20M d'indices")
ax[1].set_xlabel("Indices")
ax[1].set_ylabel("Signal Acoustique");

Une courbe du temps restant avant le prochain séisme est associée au signal accoustique en fonction de l’indice d'enregistrement. On observe qu'il y a un délai entre le signal acoustique et le temps de prévision.

### Visualisation du jeu de données de validation

On visualise les segments que l'on aura par la suite à associer avec une prédiction de temps jusqu'à un séisme.

In [ ]:
test_path = "../input/LANL-Earthquake-Prediction/test/"
test_files = listdir("../input/LANL-Earthquake-Prediction/test")
fig, ax = plt.subplots(4,1, figsize=(20,25))

for n in range(4):
    seg = pd.read_csv(test_path  + test_files[n])
    ax[n].plot(seg.acoustic_data.values, c="mediumseagreen")
    ax[n].set_xlabel("Indice")
    ax[n].set_ylabel("Signal")
    ax[n].set_ylim([-300, 300])
    ax[n].set_title("Test {}".format(test_files[n]));

L’objectif du projet est de retrouver pour chaque signal sismique le temps restant avant le prochain tremblement de terre. Pour résoudre cette problématique, nous allons entraîner notre algorithme sur le jeu de données d’apprentissage, puis nous utiliserons un jeu de validation pour le tester.

# Analyse du jeu de données

## Analyse de la taille des jeux de données

Afin d'en savoir plus sur les jeux de données que nous allons utiliser, nous nous sommes tout d'abord penchées sur l'étude de leur taille. 

### Détermination de la taille du signal d'apprentissage

On cherche à déterminer la longueur du signal sur lequel nous allons réaliser l'apprentissage.

In [ ]:
train = pd.read_csv("../input/LANL-Earthquake-Prediction/train.csv",
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
print(len(train))

Ce jeu de données a une longueur de 629 145 480 indices.

### Détermination de la taille des segments de test

On cherche à déterminer la longueur des segments qui nous sont donnnés en tant que test.

In [ ]:
test_path = "../input/LANL-Earthquake-Prediction/test/"

test_files = listdir("../input/LANL-Earthquake-Prediction/test")


seg1 = pd.read_csv(test_path  + test_files[1])
seg2 = pd.read_csv(test_path  + test_files[1])
seg3 = pd.read_csv(test_path  + test_files[1])
seg4 = pd.read_csv(test_path  + test_files[1])

print ("La durée du premier segment enregistré est de : ",len(seg1),"indices","\n","La durée moyenne des 4 premiers segments enregistrés est de : ",(len(seg1)+len(seg2)+len(seg3)+len(seg4))/4, "indices")

On constate que la taille des segments à associer est de 150 000 indices pour chaque segment test.

### Recherche du temps entre 2 séismes

Nous avons cherché à trouver le temps qui séparait deux séismes dans le fichier de données, malheureusement le fichier étant trop lourd, nous n'avons pas réussi à charger des portions assez grandes pour déterminer cette valeur, le programme que nous avions écrit figure cependant dans la case suivante. Nous pouvons tout de même constater que l'écart entre les séismes est significativement supérieur à 40 millions d'indices d'acquisition ce qui est très significativement plus grand que la taille de nos échantillons test.

In [ ]:
%list_i_eq=np.zeros(20) 
%eq=0 
%for i in range (len(train.index.values)-1):
%    if train.quaketime.values[i]==0:
%        list_i_eq[eq]=i
%        eq+=1
%print(list_i_eq)

**Visualisation des données : zoom sur le signal temporel**

In [ ]:
train = pd.read_csv("../input/LANL-Earthquake-Prediction/train.csv", nrows=20000000,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})#lecture des 20M premiers indices pour les observations afin que ce soit plus facile à ouvrir
train.head(5)

train.rename({"acoustic_data": "signal", "time_to_failure": "quaketime"}, axis="columns", inplace=True)
train.head(5)
#Les valeurs sont placées dans un tableau où sont indexés le signal et le quaketime, a savoir le temps avant le prochain tremblement de terre, pour chacun des indices d'acquisition

fig, ax = plt.subplots(3,1,figsize=(20,18))
ax[0].plot(train.index.values[0:50000], train.quaketime.values[0:50000], c="Red")
ax[0].set_xlabel("Indice")
ax[0].set_ylabel("Temps jusqu'au prochain tremblement de terre")
ax[0].set_title("A quoi ressemble le signal du temps jusqu'au prochain tremblement de terre à plus petite échelle")
ax[1].plot(train.index.values[0:49999], np.diff(train.quaketime.values[0:50000]))
ax[1].set_xlabel("Indice")
ax[1].set_ylabel("Différence entre deux indices du signal")
ax[1].set_title("Les sauts sont-ils les mêmes?")
ax[2].plot(train.index.values[0:4000], train.quaketime.values[0:4000])
ax[2].set_xlabel("Indices de 0 à 4000")
ax[2].set_ylabel("Temps jusqu'au tremblement de terre")
ax[2].set_title("Comment la prédiction temporelle varie lors de la première marche?");


À l’échelle de temps d’un séisme, c’est-à-dire environ quelques dizaines de millions d'indices, la courbe représentant le temps restant avant le prochain tremblement de terre est en dents de scie, autrement dit elle est constituée de portions de courbes affines.

Toutefois, si on passe à une échelle de temps bien inférieure telle qu’une fenêtre de taille 10 000 index, on observe que les pentes décroissantes sont en réalités constituées de sauts discontinus.

En étudiant le problème, on a pu comprendre que ces sauts étaient dus à la necessité de mettre en mémoire les informations pendant un certain temps ce qui entraine un arrêt périodique de l'acquisition. 

En supposant que le pas des sauts est à peu près constant, nous cherchons l’indice du premier saut après l’apparition du séisme.

### Recherche de la longueur entre 2 sauts dans le signal temporel


In [ ]:
cran=np.diff(train.quaketime.values[0:10000])
for i in range (8000):
    if cran[i]<-0.0008: 
        a=i
print("Les sauts sont tous les",a,"indices")
    

Sur la pente décroissante de la courbe représentant le temps restant avant le prochain tremblement de terre en fonction de l’index, l’occurrence des sauts est de 4 094 indices.

### Bilan de l'analyse du jeu de données
Rappelons que l’objectif du projet est de connaître le temps restant avant le prochain séisme pour chacun des enregistrements sismiques constituant le jeu de données de test. Pour des raisons pratiques, nous ferons par la suite le choix que l’algorithme soit conçu pour apprendre sur des sous jeux de données de même taille que celui de test.

D’après la sous-partie 1), la taille du jeu de test est de 150 000 index. Nous découpons donc le jeu de données (le signal continue) en jeux d'apprentissages (sous-signaux) de taille de 150 000 index.

### Préparation des sous jeux de données

Le jeu d’apprentissage étant constitué de petites parties du signal continu, il est nécessaire de lui associer un temps avant le prochain tremblement de terre. On le choisit de telle sorte qu’il corresponde au temps avant séisme de l’index le plus élevé de notre segment. Ainsi chaque sous jeu de données acoustiques est associé à un temps unique. Ce choix est fait en considérant qu'il existe une différence négligeable entre le temps jusqu'au prochain séisme du début du signal et celui de la fin du signal, le sous-signal étant d'une taille faible. 


Ce vecteur contenant les temps avant le prochain séisme pour chaque segment de signal sismique est utile pour la comparaison avec celle donnée par le modèle. Comme nous le verrons dans la partie suivante, l’algorithme apprend à l’aide du modèle de prévision sur le jeu de données et de la vérification, en minimisant l'écart entre ces deux valeurs.

### Découpage des signaux d'apprentissage

On choisit n signaux pour l'apprentissage de taille 150000.
On a n>nrows/150000.
On ne coupe alors pas à la suite les signaux mais on prend des morceaux qui se superposent pour pouvoir apprendre de manière plus précise.
On traites les n signaux indépendamment en implémentant une matrice de paramètres.

On décide de ne pas charger toutes les données mais seulement de prendre à chaque fois le découpage du signal qui nous intéresse, dans un soucis d'utilisation de la mémoire allouée et donc de faisabilité.

## Comment effectuer le découpage?

On sélectionne directement à la lecture les indices qui nous intéressent, de cette manière on ne garde jamais en mémoire les sous-signaux précédemment traités.

In [ ]:
#Ouverture d'un sous-signal commençant à l'indice "ind" et de longueur "taille"
ind=10 #index du début de signal d'intérêt
taille=20 #taille du signal
rownumberList=[i for i in range (ind,ind+taille)]
train = pd.read_csv("../input/LANL-Earthquake-Prediction/train.csv", skiprows=ind, nrows=taille,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
print(train)

## Paramètres choisis pour l'étude du signal

Afin de caractériser une « petite partie » de signal et lancer l'apprentissage, il nous faut choisir des paramètres de caractérisation. 

Les paramètres que nous avons choisi sont les suivants: 

- min, max, l'écart-type, la moyenne 

- max-min 

- somme

- quantile (5/95, 25/75)

- TF : même paramètres sur la TF du signal



Pour l’instant, nous mettons tous les paramètres trouvés pertinents quitte à ce que le coefficient de la régression soit faible (proche de 0).


On crée alors une fonction permettant de stocker tous ces paramètres calculés dans une liste. Il s'agit de la fonction suivante : **signal_parameters**

In [ ]:
def signal_parameters(ind,taille):
    parameters_matrix=np.zeros(20)
    signal = pd.read_csv("../input/LANL-Earthquake-Prediction/train.csv", skiprows=ind, nrows=taille,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
    signal_fft=np.real(np.fft.fft(signal)[:,0])
    parameters_matrix[0]=np.min(signal)[0]
    parameters_matrix[1]=np.max(signal)[0]
    parameters_matrix[2]=np.mean(signal)[0]
    parameters_matrix[3]=np.std(signal)[0]
    parameters_matrix[4]=parameters_matrix[1]- parameters_matrix[0]
    parameters_matrix[5]=np.sum(signal)[0]
    parameters_matrix[6]=np.quantile(signal,0.25)
    parameters_matrix[7]=np.quantile(signal,0.75)
    parameters_matrix[8]=np.quantile(signal,0.05)
    parameters_matrix[9]=np.quantile(signal,0.95)
    parameters_matrix[10]=np.min(signal_fft)
    parameters_matrix[11]=np.max(signal_fft)
    parameters_matrix[12]=np.mean(signal_fft)
    parameters_matrix[13]=np.std(signal_fft)
    parameters_matrix[14]=parameters_matrix[11]- parameters_matrix[10]
    parameters_matrix[15]=np.sum(signal_fft)
    parameters_matrix[16]=np.quantile(signal_fft,0.25)
    parameters_matrix[17]=np.quantile(signal_fft,0.75)
    parameters_matrix[18]=np.quantile(signal_fft,0.05)
    parameters_matrix[19]=np.quantile(signal_fft,0.95)
    time_to_failure=np.min(signal)[1]
    
    return parameters_matrix,time_to_failure

Maintenant que nous avons créé une fonction permettant d'obtenir et de stocker tous les paramètres d'un signal, il suffit de l'utiliser pour obtenir les paramètres de tous nos sous-signaux d'apprentissage et les stocker pour ensuite apprendre. 

In [ ]:
n=100 #nombre de signaux d'apprentissage, pour 100 signaux, le programme met environ 10 min à tourner
parameters = []
size = 150000
pas=size//2
time_to_failure = []
for i in range (n):
    ind=i*pas
    A,B=signal_parameters(ind,size)
    parameters.append(A)
    time_to_failure.append(B)


**AFFICHAGE DES 5 PREMIERS PARAMETRES** 

On affiche les 5 premiers signaux d'apprentissage c'est à dire le minimum, le maximum, la std, la moyenne et la différence min-max, ainsi que le temps jusqu'au prochain séisme associé.

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(20,18))
plt.subplot(321)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Minimum");
plt.plot([x[0] for x in parameters])
plt.subplot(322)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Maximum");
plt.plot([x[1] for x in parameters])
plt.subplot(323)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Ecart type");
plt.plot([x[2] for x in parameters])
plt.subplot(324)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Moyenne");
plt.plot([x[3] for x in parameters])
plt.subplot(325)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Maximum-Minimum");
plt.plot([x[4] for x in parameters])
plt.subplot(326)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètre")
plt.title("Temps jusqu'au prochain tremblement de terre");
plt.plot(time_to_failure)

# Début de l'apprentissage (régression linéaire)
## Premier essai

Input = Signal accoustique

Output = Time to failure

On utilise un modèle de régression linéaire pour réaliser l'apprentissage à l'aide de nos paramètres calculés.

In [ ]:
import torch 
from torch.autograd import Variable 
  
x_data = Variable(torch.Tensor(parameters)) 
y_data = Variable(torch.Tensor(time_to_failure)) 


In [ ]:
class LinearRegressionModel(torch.nn.Module): 
  
    def __init__(self): 
        super(LinearRegressionModel, self).__init__() 
        self.linear = torch.nn.Linear(20, 1)  
  
    def forward(self, x): 
        y_pred = self.linear(x) 
        return y_pred 
    
lr=0.001
our_model = LinearRegressionModel()   
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(our_model.parameters(), lr) 
  
for epoch in range(5000): 
    optimizer.zero_grad() 
    pred_y = our_model(x_data) 
    loss = criterion(pred_y, y_data) 
    loss.backward() 
    optimizer.step() 
    print('epoch {}, loss {}'.format(epoch, loss.item())) 
  


**ERREUR**
Ce plan d'apprentissage ne fonctionne pas, on peut observer un problème au niveau des données, la loss donne NaN. Cela est probablement dû a des divisions 0/0 mal gérées, c'est pour cela qu'on observe une divergence avant d'obtenir les NaN. 

On va essayer de transformer les données en les normalisant afin d'éviter ce problème.

Pour cela on utilise un module de normalisation déjà codé qui permet de normaliser les caractéristiques en supprimant la moyenne et en mettant à l'echelle la variance unitaire. Les valeurs sont calculées comme z=(x-u)/s avec u la moyenne et s l'écart type des échantillons d'apprentissage.

### Normalisation du signal

In [ ]:
#Normalisation du signal
from sklearn.preprocessing import StandardScaler 
sc = StandardScaler()

parameters_fit=sc.fit_transform(parameters)
x_data_fit=Variable(torch.Tensor(parameters_fit)) 

### Affichage des paramètres fits  (normalisés)

On affiche les paramètres une fois normalisés ainsi que les temps jusqu'au prochain séisme associé.

In [ ]:
fig, ax = plt.subplots(3,2,figsize=(20,18))
plt.subplot(321)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Minimum");
plt.plot([x[0] for x in parameters_fit])
plt.subplot(322)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Maximum normalisé");
plt.plot([x[1] for x in parameters_fit])
plt.subplot(323)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Ecart type normalisé");
plt.plot([x[2] for x in parameters_fit])
plt.subplot(324)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Moyenne  normalisée");
plt.plot([x[3] for x in parameters_fit])
plt.subplot(325)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Maximum-Minimum normalisé");
plt.plot([x[4] for x in parameters_fit])
plt.subplot(326)
plt.xlabel("Signal d'apprentissage")
plt.ylabel("Paramètres")
plt.title("Temps jusqu'au prochain tremblement de terre");
plt.plot(time_to_failure)

## Deuxième essai de Regression linéaire

In [ ]:
lr=0.01
our_model = LinearRegressionModel()   
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(our_model.parameters(), lr) 
  
for epoch in range(5000): 
    optimizer.zero_grad() 
    pred_y = our_model(x_data_fit) 
    loss = criterion(pred_y, y_data) 
    loss.backward() 
    optimizer.step() 
    print('epoch {}, loss {}'.format(epoch, loss.item())) 
  


On observe désormais une loss qui fonctionne, il y a bien apprentissage via le jeu de paramètres.

## Test du modèle sur un jeu de validation

On crée un jeu de données de validation à l'aide du signal d'entrée, pour cela on découpe le signal selon un autre sous jeux de données que celui initialement défini pour l'apprentissage. Puis on test le modèle obtenu sur ce jeu de données.

In [ ]:
n_valid=100 #nombre de signaux de validation
parameters_valid = []
size = 150000
pas=size//2
time_to_failure_valid = []
pas_initial=1000000 #pour ne pas utiliser le même jeu que l'apprentissage
for i in range (n_valid):
    ind=i*pas+pas_initial
    A,B=signal_parameters(ind,size)
    parameters_valid.append(A)
    time_to_failure_valid.append(B)

In [ ]:
x_data_valid_fit = Variable(torch.Tensor(sc.fit_transform(parameters_valid))) 
y_data_valid = Variable(torch.Tensor(time_to_failure_valid)) 
pred_y_valid = our_model(x_data_valid_fit) 
loss = criterion(pred_y_valid, y_data_valid) 

print('loss {}'.format(loss.item()))

Pour pouvoir observer l'efficacité de notre modèle, on compare les valeurs obtenues à l'aide de ce dernier et les valeurs données par le jeu de validation, c'est ce que l'on execute dans la cellule suivante. 

In [ ]:
plt.plot(pred_y_valid.detach().numpy(),'+')
plt.plot(time_to_failure_valid)
plt.legend(["données d'apprentissage", "données de validation"], loc ="upper left") 
plt.xlabel("nombre d'époques")
plt.ylabel("Temps jusqu'au prochain tremblement de terre")
plt.title("Visualisation de l'éfficacité du modèle");

Nous n’avons réussi à faire tourner le programme qu’avec un nombre de signaux d’apprentissages de 100. Avec 100 signaux, nous n’observons même pas de tremblement de terre et les temps avant tremblement de terre ne sont pas bien balayés, ils sont même assez proches. Cependant, on observe déjà des temps de calcul des paramètres beaucoup trop longs et qui saturent la CPU de kaggle, bloquant l’exécution du programme. Nous n’arrivons donc pas à des résultats aussi satisfaisants que ce que l’on pourrait espérer.

### Pistes d’améliorations
Pour améliorer ce programme, on pourrait faire une sélection plus fine des paramètres à prendre en compte. On pourrait s’intéresser à la corrélation de ces paramètres avec le temps avant le prochain tremblement. Partir d’un modèle prenant en compte le paramètre le plus corrélé, on peut ensuite ajouter des paramètres supplémentaires en effectuant un test de comparaison de modèles gigognes. De cette manière, nous obtiendrons une meilleure sélection des paramètres.

Une autre piste serait aussi de jouer sur le pas d’apprentissage et de le faire varier pour voir son impact sur l’apprentissage.

Ces différents points pourraient être intéressants à explorer mais comme nous avons des difficultés pour faire tourner le programme, nous décidons de ne pas aller plus loin. Même si nous ne savons pas si notre modèle (sur un plus grand nombre de signaux d’apprentissage) est efficace, ce projet nous a permis de mieux nous familiariser avec l’environnement collab, le code lié à l’apprentissage profond, mais surtout l’importance du traitement des données. On a notamment pu constater que celui-ci demandait considérablement plus de temps en comparaison à l’écriture du programme d’apprentissage.